# Part 1 : Data Preprocessing

## 1. Introduction

To have a good model, we need good data (remember "garbage in, garbage out") This notebook goes through a necessary step of **data cleaning** and the cleaned data will be organized into a **Corpus and Document -Term Matrix**) so that it is easy to use in machine learning algorithms.

### 1.1. Problem Statement

We are looking at transcripts of some stand up comedians and note their similarities and differences. After cleaning those transcripts will be used to train diverse machine learning models.

### 1.2. Data

The data use in this project are scraped stand-up comedy transcripts of 12 comedians from  [Scraps From The Loft](http://scrapsfromtheloft.com). **scrapsfromtheloft.com** is a website serving as a cultural multilingual magazine featuring movie reviews and essays, stand-up comedy transcripts, interviews, great literature, history, and many more. Selection of the 12 comedians was based on release year (past 5 years), rating level (greater than a 7.5/10)  and vote (more than 2000) according to [IMBD](https://www.imdb.com/).

* **Source :** [Scraps From scrapsfromtheloft](http://scrapsfromtheloft.com)

### 1.4.  Import Libraries

In [1]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle
import re
import string
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 2. Getting The Data

### 2.1. Scraping

In [2]:
# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    #text = [p.text for p in soup.find(class_="post-content").find_all('p')] #page html code was modified
    text = [p.text for p in soup.find_all('p')]
    #print(url)
    return text

In [3]:
# URLs of transcripts in scope
urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
        'http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/'
        
       ]
# Comedian names
comedians = ['louis', 'dave', 'ricky', 'bo', 'bill', 'jim', 'john', 'hasan', 'ali', 'anthony', 'mike', 'joe']

### 2.2. Extracting transcripts
> It takes a while depending on your internet connexion speed ...

In [4]:
transcripts = [url_to_transcript(u) for u in urls]

### 2.3. Save Extracted transcripts`
> save transcripts as csv

In [8]:
# using naive method to convert lists to dictionary
data = {}
for key in comedians:
	for value in transcripts:
		data[key] = value
		transcripts.remove(value)
		break

In [17]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [18]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [25]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df.to_csv('../data/transcripts_v1.csv')
data_df

,transcript
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ..."
anthony,"Thank you. Thank you. Thank you, San Francisco. Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my s..."
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ..."
bo,© 2021 Scraps from the Loft. All rights reserved.
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
hasan,© 2021 Scraps from the Loft. All rights reserved.
jim,"[Car horn honks] [Audience cheering] [Announcer] Ladies and gentlemen, please welcome to the stage Mr. Jim Jefferies! [Upbeat music playing] Hello..."
joe,"[rock music playing] [audience cheering] [announcer] Ladies and gentlemen, welcome Joe Rogan. [audience cheering and applauding] What the f*ck is ..."
john,"Armed with boyish charm and a sharp wit, the former “SNL” writer John Mulaney offers sly takes on marriage, his beef with babies and the time he m..."
louis,Intro\nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily a...


## 3. Data Cleaning

> Text cleaning consist of Make text all lower case, Remove punctuation, Remove numerical values, Remove common, non-sensical text (/n), Tokenize text and Remove stop words.

> Text cleaning is not one time process it can be done many times as it needed to improve the results. **More data cleaning steps after tokenization:** Stemming / lemmatization, Parts of speech tagging, Create bi-grams or tri-grams
Deal with typos And more...

In [27]:
def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets, remove punctuation,
        remove words containing numbers and non-sensical text.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text


In [28]:
transcript_cleaned = lambda x: clean_text(x)

In [30]:
# S
data_clean = pd.DataFrame(data_df.transcript.apply(transcript_cleaned))
data_clean.to_csv('../data/transcripts_v2.csv')

> **NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:

- Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
- Combine 'thank you' into one term (bi-grams)
- And a lot more...

# 4. Organizing The Data
1. **Corpus :** a collection of text
2. **Document-Term Matrix :** word counts in matrix format

## 4.1. Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [ ]:
# Let's take a look at our dataframe
#data_df

In [32]:
# Let's add the comedians' full names as well
full_names = ['Ali Wong', 'Anthony Jeselnik', 'Bill Burr', 'Bo Burnham', 'Dave Chappelle', 'Hasan Minhaj',
              'Jim Jefferies', 'Joe Rogan', 'John Mulaney', 'Louis C.K.', 'Mike Birbiglia', 'Ricky Gervais']
data_df['full_name'] = full_names

# Let's pickle it for later use
data_df.to_pickle("../data/corpus.pkl")
data_df.to_csv("../data/corpus.csv")

data_df

,transcript,full_name
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ...",Ali Wong
anthony,"Thank you. Thank you. Thank you, San Francisco. Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my s...",Anthony Jeselnik
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ...",Bill Burr
bo,© 2021 Scraps from the Loft. All rights reserved.,Bo Burnham
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ...",Dave Chappelle
hasan,© 2021 Scraps from the Loft. All rights reserved.,Hasan Minhaj
jim,"[Car horn honks] [Audience cheering] [Announcer] Ladies and gentlemen, please welcome to the stage Mr. Jim Jefferies! [Upbeat music playing] Hello...",Jim Jefferies
joe,"[rock music playing] [audience cheering] [announcer] Ladies and gentlemen, welcome Joe Rogan. [audience cheering and applauding] What the f*ck is ...",Joe Rogan
john,"Armed with boyish charm and a sharp wit, the former “SNL” writer John Mulaney offers sly takes on marriage, his beef with babies and the time he m...",John Mulaney
louis,Intro\nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily a...,Louis C.K.


## 4.2. Document-Term Matrix
+ Tokenization
+ remove stop words

In [34]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = data_clean.index
data_dtm.to_pickle("../data/dtm.pkl")
data_dtm.to_csv("../data/dtm.csv")
pickle.dump(cv, open("../data/cv.pkl", "wb"))
data_dtm

aaaaah  aaah  aah  abc  abcs  ability  abject  able  ablebodied  \
ali           0     0    0    1     0        0       0     2           0   
anthony       0     0    0    0     0        0       0     0           0   
bill          1     0    0    0     1        0       0     1           0   
bo            0     0    0    0     0        0       0     0           0   
dave          0     1    0    0     0        0       0     0           0   
hasan         0     0    0    0     0        0       0     0           0   
jim           0     0    0    0     0        0       0     1           2   
joe           0     0    0    0     0        0       0     2           0   
john          0     0    0    0     0        0       0     3           0   
louis         0     0    3    0     0        0       0     1           0   
mike          0     0    0    0     0        0       0     0           0   
ricky         0     0    0    0     0        1       1     2           0   

         abortion  abortions  abroad  absolute  absolutely  absorb  absorbing  \
ali             0          0       0         0           0       0          1   
anthony         2          0       0         0           0       0          0   
bill            0          0       0         1           3       1          0   
bo              0          0       0         0           0       0          0   
dave            0          1       0         0           0       0          0   
hasan           0          0       0         0           0       0          0   
jim             0          0       3         0           0       0          0   
joe             0          0       0         0           0       0          0   
john            0          0       0         0           1       0          0   
louis           0          0       0         0           0       0          0   
mike            0          0       0         0           0       0          0   
ricky           0          0       0         0           0       0          0   

         abuse  abused  accent  accents  accept  acceptable  accepted  access  \
ali          1       2       0        0       0           0         0       0   
anthony      0       0       1        0       0           0         0       0   
bill         0       0       0        0       0           1         0       0   
bo           0       0       0        0       0           0         0       0   
dave         0       0       0        0       0           0         0       0   
hasan        0       0       0        0       0           0         0       0   
jim          0       0       4        0       0           0         0       0   
joe          1       0       1        1       0           0         1       0   
john         0       0       1        0       0           1         0       2   
louis        0       0       0        0       1           0         0       2   
mike         0       0       0        0       0           1         0       0   
ricky        0       0       0        0       0           0         0       0   

         accident  accidentally  accomplished  according  accoutrements  \
ali             1             1             0          2              0   
anthony         0             1             0          0              0   
bill            0             2             0          0              0   
bo              0             0             0          0              0   
dave            0             0             0          0              0   
hasan           0             0             0          0              0   
jim             0             0             0          0              0   
joe             0             1             0          0              0   
john            0             0             1          0              1   
louis           1             0             0          0              0   
mike           15             0             0          0              0   
ricky       

# 5. Save cleaned data

In [35]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
pickle.dump(cv, open("../data/cv.pkl", "wb"))
data_clean.to_pickle('../data/data_clean.pkl')
data_clean.to_csv('../data/data_clean.csv')

---
---

* **Additional Exercises**
    1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
    2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?
    3. There are definitely some things that could be better cleaned up, such as adding more stop words or including bi-grams.

* **References**
    * [nlp-in-python-tutorial by Alice Zhao](https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb)

    * [Natural Language Processing (NLP)and CountVectorizer by Sanjay Singh](https://medium.com/sanrusha-consultancy/natural-language-processing-nlp-and-countvectorizer-5571cf9205e4#:~:text=ngram_range%20indicates%20how%20many%20words%20will%20be%20considered,Run%20below%20line%20of%20code%20to%20understand%20how)
    * [How to Scrape Content from a Website by FabianOmoke](https://www.studytonight.com/post/how-to-scrape-content-from-a-website-using-beautifulsoup-python)